<a href="https://colab.research.google.com/github/MartinPG22/Tecnicas-Aprendizaje-Automatico/blob/main/P1_FINAL_MODEL_100472279_100472236.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicción con el Modelo Final de la Producción de Energía Eólica** #


---


G1:

    Martín Portugal González: 100472279

    Alba Vidales Casado: 100472236

Enlace repositorio: https://github.com/albavidales/Aprendizaje-Automatico-p1-100472279-100472236.git


---




## 1. Preparación de los datos ##

Antes de entrenar nuestro modelo, tenemos que preparar nuestros datos

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import math

En primer lugar, y de acuerdo con el EDA realizado en el primer *notebook*, eliminaremos las variables que no se correspondan con la localización de Sotavento y las variables que decidimos que no aportaban nada relevante a nuestro dataset.

Además, partimos los datos en *train* y *test* para poder entrenar nuestro modelo y hacer una estimación de su funcionamiento.

In [ ]:
nombre_archivo = "wind_ava.csv.gz"
dataset = pd.read_csv(nombre_archivo, sep = ',')

columnas_drop= [col for col in dataset.columns if not (col.endswith('.13') or col == "datetime" or col == "energy")]
dataset = dataset.drop(columns=columnas_drop)

del dataset['lai_lv.13']
del dataset['u10.13']
del dataset['v10.13']

dataset['Fecha'] = pd.to_datetime(dataset['datetime'])
dataset['Año'] = dataset['Fecha'].dt.year
dataset['Mes'] = dataset['Fecha'].dt.month
dataset.set_index('Fecha', inplace=True)

del dataset['datetime']

dataset.drop(columns=['Año'])

train_data = dataset.loc['2005-01-01':'2007-12-31']  # Datos de 2005, 2006 y 2007
test_data = dataset.loc['2008-01-01':'2009-12-31']   # Datos de 2008 y 2009

X_train = train_data.drop(columns=["energy"])
y_train = train_data["energy"]
X_test = test_data.drop(columns=["energy"])
y_test = test_data["energy"]

## 2. Entrenamiento del modelo ##

Después de preparar nuestro conjunto de datos, estamos listos para entrenar nuestro modelo utilizando los datos de entrenamiento del archivo *wind_ava.csv.gz*. Además, evaluaremos el rendimiento futuro de nuestro modelo, lo cual nos permitirá comprender cómo se desempeñará el modelo en datos no vistos, proporcionando información crucial sobre su capacidad para generalizar y su utilidad.

Es importante recordar que el modelo que vamos a utilizar es el que se seleccionó como mejor modelo en el primer notebook, en este caso KNN. Igualmente, seguiremos utilizando el que resultaba como mejor scaler (*StandarScarler*) y los hiperparámetros obtenidos en el proceso de HPO (*k=16*, *weights='distance'*, *p=1*).

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# Definir el pipeline con el escalador y el clasificador KNN
modelo_pre_final = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor(n_neighbors=16, weights='distance', p=1))  # Clasificador KNN con los hiperparámetros deseados
])

# Inicializar TimeSeriesSplit para la validación cruzada
tscv = TimeSeriesSplit(n_splits=3)

# Realizar la validación cruzada
for train_index, val_index in tscv.split(X_train):
    X_train_inner, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_inner, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Entrenar el modelo en el conjunto de entrenamiento interno
    modelo_pre_final.fit(X_train_inner, y_train_inner)

# Entrenar el modelo final en todo el conjunto de entrenamiento
start_time = time.time()
modelo_pre_final.fit(X_train, y_train)
training_time = time.time() - start_time

# Realizar predicciones en el conjunto de prueba
y_pred = modelo_pre_final.predict(X_test)

# Calcular métricas de evaluación en el conjunto de prueba
mae = mean_absolute_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))

print("MAE del modelo en el conjunto de prueba:", mae)
print("RMSE del modelo en el conjunto de prueba:", rmse)
print("Tiempo de entrenamiento y predicción:", training_time, "segundos")


MAE del modelo en el conjunto de prueba: 314.3529968804288
RMSE del modelo en el conjunto de prueba: 423.56930525263715
Tiempo de entrenamiento y predicción: 0.009713411331176758 segundos


Una vez obtenida la evaluación del rendimiento futuro del modelo seleccionado, procedemos a entrenarlo con todos los datos disponibles

In [ ]:

x_train_final = dataset.drop(columns=["energy"])
y_train_final = dataset["energy"]

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# Definir el pipeline con el escalador y el clasificador KNN
modelo_final = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor(n_neighbors=16, weights='distance', p=1))  # Clasificador KNN con los hiperparámetros deseados
])

# Inicializar TimeSeriesSplit para la validación cruzada
tscv = TimeSeriesSplit(n_splits=3)

# Realizar la validación cruzada
for train_index, val_index in tscv.split(x_train_final):
    X_train_inner, X_val = x_train_final.iloc[train_index], x_train_final.iloc[val_index]
    y_train_inner, y_val = y_train_final.iloc[train_index], y_train_final.iloc[val_index]

    # Entrenar el modelo en el conjunto de entrenamiento interno
    modelo_final.fit(X_train_inner, y_train_inner)

# Entrenar el modelo final en todo el conjunto de entrenamiento
modelo_final.fit(x_train_final, y_train_final)

## 3. Datos de Competición ##


Antes de realizar las predicciones, necesitamos preparar los datos para poder introducirlos en nuestro modelo ya entrenado. Obtenemos los datos de competición del archivo *wind_comp.csv.gz* y realizamos la limpieza de las localizaciones que no son Sotavento, como siempre. Lo vemos en las siguientes líneas de código:

In [ ]:
nombre_archivo = "wind_comp.csv.gz"
dataset2 = pd.read_csv(nombre_archivo, sep = ',')

columnas_drop= [col for col in dataset2.columns if not (col.endswith('.13') or col == "datetime" or col == "energy")]
dataset2 = dataset2.drop(columns=columnas_drop)

del dataset2['lai_lv.13']
del dataset2['u10.13']
del dataset2['v10.13']

dataset2['Fecha'] = pd.to_datetime(dataset2['datetime'])
dataset2['Año'] = dataset2['Fecha'].dt.year
dataset2['Mes'] = dataset2['Fecha'].dt.month
dataset2.set_index('Fecha', inplace=True)

del dataset2['datetime']
dataset2.drop(columns=['Año'])

,p54.162.13,p55.162.13,cape.13,p59.162.13,lai_hv.13,u10n.13,v10n.13,sp.13,stl1.13,t2m.13,stl2.13,stl3.13,iews.13,inss.13,stl4.13,fsr.13,flsr.13,u100.13,v100.13,Mes
Fecha,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,2.380345e+06,10.537942,10.382510,2.220341e+06,2.433955,1.258152,4.389059,96056.601316,278.713081,277.372035,280.494544,282.003767,0.223763,0.469945,285.865293,0.426499,-5.692383,3.028044,6.801977,1
2010-01-01 06:00:00,2.387186e+06,11.038628,1.136771,3.593417e+06,2.433821,0.258600,-0.063731,96132.834569,278.298853,276.877148,280.011630,281.986895,0.049144,0.037347,285.844043,0.426720,-5.673355,1.905932,0.819461,1
2010-01-01 12:00:00,2.411521e+06,9.635489,0.265247,4.477477e+06,2.433676,4.505959,1.314488,96882.102158,280.531223,281.085733,279.880559,281.958683,0.363838,0.075026,285.823566,0.420984,-5.772643,6.254707,1.658140,1
2010-01-01 18:00:00,2.423920e+06,10.724937,4.281838,6.237006e+06,2.433542,1.620416,0.278042,97243.861285,279.684007,278.417187,280.376232,281.945406,0.152444,0.057106,285.803281,0.418642,-5.807686,4.680087,1.276968,1
2010-01-02 00:00:00,2.436652e+06,13.533924,1.250448,5.666546e+06,2.433397,0.958361,1.244256,97438.059470,277.778660,276.086144,279.878626,281.929364,0.098397,0.105619,285.783963,0.419667,-5.780384,2.686441,3.517605,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-30 18:00:00,2.427100e+06,15.771087,0.000000,3.854556e+05,2.434715,-1.055901,0.695958,96835.296716,281.603551,282.442740,281.134047,280.484451,-0.063578,0.081295,284.757585,0.395475,-6.002601,-3.337395,3.368429,12
2010-12-31 00:00:00,2.432052e+06,16.008218,0.000000,2.991796e+05,2.434570,-0.993059,0.396950,97026.958021,279.380815,279.223347,280.724595,280.559407,-0.052669,0.065760,284.736141,0.395862,-5.995068,-3.143314,2.426972,12
2010-12-31 06:00:00,2.433778e+06,16.676928,0.000000,2.998956e+05,2.434436,-0.815993,0.527332,97191.601551,278.402283,277.723528,280.083545,280.607257,-0.042174,0.057719,284.714312,0.396176,-5.977477,-2.658373,2.449922,12


## 3. Predicciones y Modelo Final

 Primero, se guarda el modelo final entrenado en un archivo utilizando la biblioteca Joblib. Este archivo, llamado '*modelo_final.pkl*', contiene todos los parámetros y configuraciones del modelo, lo que permite cargarlo fácilmente para hacer predicciones sin tener que volver a entrenar el modelo desde cero.

 Posteriormente, se emplea el modelo final para realizar predicciones sobre el conjunto de datos *dataset2* que se preparó en el apartado anterior y que contiene los datos del archivo *wind_comp.csv.gz*.

 Por último, estas predicciones se guardan en un archivo CSV llamado '*predicciones.csv*'.

In [ ]:
# Guardar el modelo final en un archivo
import joblib
joblib.dump(modelo_final, 'modelo_final.pkl')

# Hacer predicciones para el conjunto de datos de la competición
predicciones_competicion = modelo_final.predict(dataset2)

# Guardar las predicciones en un archivo CSV
import pandas as pd
predicciones_df = pd.DataFrame(predicciones_competicion, columns=['prediccion'])
predicciones_df.to_csv('predicciones.csv', index=False)